In [4]:
import openai
from qdrant_client import QdrantClient

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference data point from LangSmith

In [5]:
client = Client()

In [6]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [7]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('f64321e5-0d4c-405e-b95a-e97c3f02d756'), created_at=datetime.datetime(2026, 1, 20, 6, 7, 32, 494141, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 20, 6, 7, 32, 494141, tzinfo=TzInfo(0)), example_count=35, session_count=8, last_session_start_time=datetime.datetime(2026, 1, 20, 20, 44, 47, 75065), inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'Linux-6.8.0-90-generic-x86_64-with-glibc2.39', 'sdk_version': '0.6.4', 'runtime_version': '3.12.3', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [8]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'Yes. The DOBEWINGDELOU PS5 glossy faceplate (B0BKTH61S2) mentions a cooling vents design that helps promote air circulation to reduce overheating and extend console life.',
 'reference_context_ids': ['B0BKTH61S2'],
 'reference_descriptions': ["PS5 Faceplate with RGB Light Strip DOBEWINGDELOU PS5 Glossy Face Plate 400+ Effects LED Light Ring Console Cover for PlayStation 5 Disc Edition DIY Decoration Kit Accessories Eelectroplating Black Cooling vents design: The PS5 faceplates come with vents, which can help cool down your PS5 console when in operation, better and faster promote air circulation and reduce the PS5 console overheating. So as to extend the service life of the PS5 console and give you a smoother gaming experience. Precisely tailored: the PS5 faceplates and RGB LED light is specially designed for PlayStation 5 disc version. the new shell can replace the original cover directly, and the LED light ring design makes it available to paste directly to the PS5 c

In [9]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Is there a protective faceplate that claims to help cool the PS5?'}

In [10]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

### RAG Pipeline

In [11]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [12]:
rag_pipeline("Can I get some charger?", top_k=5)

{'answer': 'Yes. The available chargers include:\n\n- B0BWQYJY87: YOXXZUS Switch Charger (39W, 15V 2.6A) with 5ft USB-C cable. Compatible with Switch OLED/Switch Lite for charging and TV dock mode. Includes protection features and 1-year replacement.\n\nIf you’d like, I can help you pick based on which device you’re charging (Nintendo Switch vs other devices) and whether you need TV-dock compatibility or extra-long cable.',
 'question': 'Can I get some charger?',
 'retrieved_context_ids': ['B0BWQYJY87',
  'B09PVCVQDP',
  'B0B1PM6D24',
  'B0B38YMM68',
  'B09Y8TFH8D'],
 'retrieved_context': ['YOXXZUS Switch Charger,Switch AC Adapter Support TV Dock Mode for Switch OLED/Switch Lite 39W (15V 2.6A) 【Safe and Stable Charging】It takes about 2.5 hours to fully charge the SWITCH,short circuit protection, over current protection, surge protection, overload protection, etc. 【Support Switch TV Dock】This charger is compatible with the SWITCH TV dock,It works for docking the switch to play on TV.(No

### RAGAS metrics

In [13]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

/tmp/ipykernel_95447/1973645927.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/tmp/ipykernel_95447/1973645927.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/tmp/ipykernel_95447/1973645927.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Fa

In [14]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/tmp/ipykernel_95447/2270190839.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/tmp/ipykernel_95447/2270190839.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [15]:
reference_input

{'question': 'Is there a protective faceplate that claims to help cool the PS5?'}

In [16]:
reference_output

{'ground_truth': 'Yes. The DOBEWINGDELOU PS5 glossy faceplate (B0BKTH61S2) mentions a cooling vents design that helps promote air circulation to reduce overheating and extend console life.',
 'reference_context_ids': ['B0BKTH61S2'],
 'reference_descriptions': ["PS5 Faceplate with RGB Light Strip DOBEWINGDELOU PS5 Glossy Face Plate 400+ Effects LED Light Ring Console Cover for PlayStation 5 Disc Edition DIY Decoration Kit Accessories Eelectroplating Black Cooling vents design: The PS5 faceplates come with vents, which can help cool down your PS5 console when in operation, better and faster promote air circulation and reduce the PS5 console overheating. So as to extend the service life of the PS5 console and give you a smoother gaming experience. Precisely tailored: the PS5 faceplates and RGB LED light is specially designed for PlayStation 5 disc version. the new shell can replace the original cover directly, and the LED light ring design makes it available to paste directly to the PS5 c

In [17]:
result = rag_pipeline(reference_input["question"])

In [18]:
result

{'answer': 'Yes. The PS5 faceplate options that mention cooling are:\n- B0BNH4CNW3: Electroplating Face Plates with Cooling Vents Cover Skins Shell Panels for PS5 Disc Edition Console — describes cooling vents designed to help protect and potentially aid cooling.\n- B0BKTH61S2: PS5 Faceplate with RGB Light Strip — mentions vents to help cool down the console and improve air circulation.\n\nSo, there are protective faceplates that claim to help with cooling.',
 'question': 'Is there a protective faceplate that claims to help cool the PS5?',
 'retrieved_context_ids': ['B0BNH4CNW3',
  'B0BKTH61S2',
  'B0B1WC6CZ2',
  'B0BQDQ5FQM',
  'B09SNXFLGG'],
 'retrieved_context': ['Electroplating Face Plates with Cooling Vents Cover Skins Shell Panels for PS5 Disc Edition Console, Playstation 5 Accessories Faceplate Protective Replacement Plate (Electroplating Gold) High-Quality Electroplating Replacement Hard Shell, Only Suitable for PS5 CD / Disc Version Console. Professional Design: Easily Access 

In [19]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [20]:
await ragas_faithfulness(result, "")

0.5

In [21]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [22]:
await ragas_responce_relevancy(result, "")

np.float64(0.8794700015334712)

In [23]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [24]:
await ragas_context_precision_id_based(result, reference_output)

0.2

In [25]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [26]:
await ragas_context_recall_id_based(result, reference_output)

1.0